This notebook is designed to generate the continious request file for all the training phases, so we can benchmark to see how many phases is correctly picked.

In [6]:
import pandas as pd
from obspy import UTCDateTime
from pathlib import Path
import json

In [2]:
df=pd.read_csv("/mnt/home/xiziyi/Packages_Research/PhaseNet-TF/dataset/phase_picks.csv")
output=Path("/mnt/home/xiziyi/Packages_Research/PhaseNet-TF/batch_inference/train_benchmark.csv")

In [3]:
df

,event_id,station_id,phase_index,phase_time,phase_score,phase_type,phase_polarity
0,11_52111,A01,9600,2009-12-02T01:14:41.317870,1,P,N
1,11_52111,A01,10001,2009-12-02T01:14:51.343700,1,S,N
2,11_52111,A02W,9600,2009-12-02T01:14:42.457750,1,P,N
3,11_52111,A02W,10045,2009-12-02T01:14:53.596650,1,S,N
4,11_52111,A03,9600,2009-12-02T01:14:43.547000,1,P,N
...,...,...,...,...,...,...,...
60302,4_56400,TNGA,10026,2010-10-26T01:01:40.717519,1,PS,N
60303,4_56400,VABL,9600,2010-10-26T01:01:32.764610,1,P,N
60304,4_56400,VAVP,9600,2010-10-26T01:01:38.015000,1,P,N
60305,4_56400,VAVP,11732,2010-10-26T01:02:31.333830,1,S,N


In [9]:
mapper={}
with open("/mnt/home/xiziyi/Packages_Research/PhaseNet-TF/dataset/stations.json","r") as f:
    json_raw=json.load(f)
for key in json_raw.keys():
    net,sta,_,_=key.split(".")
    if sta not in mapper:
        mapper[sta]=net
    else:
        assert mapper[sta]==net

In [12]:
time_collection=set()
with output.open("w") as f:
    f.write("network,station,start_time,end_time\n")
    for i in range(len(df)):
        row=df.iloc[i]
        t=UTCDateTime(row.phase_time)
        rounded_start=UTCDateTime(f"{t.year}-{t.month}-{t.day}T{t.hour}")
        rounded_end=rounded_start+60*60
        time_collection.add(f"{mapper[row.station_id]},{row.station_id},{str(rounded_start)},{str(rounded_end)}\n")
    for each in sorted(time_collection):
        f.write(each)